In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [64]:
#計測地点のosm_idを指定
start_osm = 211152186
mid_osm = [214849323,211152190]
end_osm =  51396950
start_mesh = 50325642424
end_mesh = 50325642222

In [65]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [66]:
getdbname = "kddi_06_new_param"
dbname = "kddi_trkn06_monthly_anbun"
trknName = "trkn_06_grp"
sttime = "2022-04-01 06:00:00"

In [67]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値14.116666666666667距離0.0 km
50325642422
存在メッシュ：50325642422値16.054285997594057距離0.11554016109201981 km
50325642242
存在メッシュ：50325642242値19.929524606474715距離0.34662048011721447 km
50325642244
存在メッシュ：50325642244値17.991905930902334距離0.2310803581039872 km
50325642422
存在メッシュ：50325642422値16.054285997594057距離0.11554016109201981 km
50325642424
存在メッシュ：50325642424値14.116666666666667距離0.0 km
50325642224
存在メッシュ：50325642224値21.86714295441504距離0.46216058259375614 km
50325642242
存在メッシュ：50325642242値19.929524606474715距離0.34662048011721447 km
50325642224
存在メッシュ：50325642224値21.86714295441504距離0.46216058259375614 km
50325642224
存在メッシュ：50325642224値21.86714295441504距離0.46216058259375614 km
50325642222
存在メッシュ：50325642222値23.804761904761904距離0.5777007209917748 km
50325642242
存在メッシュ：50325642242値19.929524606474715距離0.34662048011721447 km
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値19.633

33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値83.0距離0.0 km
50325642422
50325642422 is kddi none!
50325642242
存在メッシュ：50325642242値122.72941721166421距離0.34662048011721447 km
50325642244
存在メッシュ：50325642244値109.48628249960736距離0.2310803581039872 km
50325642422
50325642422 is kddi none!
50325642424
存在メッシュ：50325642424値83.0距離0.0 km
50325642224
存在メッシュ：50325642224値135.97254968443866距離0.46216058259375614 km
50325642242
存在メッシュ：50325642242値122.72941721166421距離0.34662048011721447 km
50325642224
存在メッシュ：50325642224値135.97254968443866距離0.46216058259375614 km
50325642224
存在メッシュ：50325642224値135.97254968443866距離0.46216058259375614 km
50325642222
存在メッシュ：50325642222値149.2156862745098距離0.5777007209917748 km
50325642242
存在メッシュ：50325642242値122.72941721166421距離0.34662048011721447 km
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値82.35距離0.0 km
50325642422
存在メッシュ：50325642422値95.52182010773132距離0.11554016109201981 km
50325642242
存在メッシ

33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
50325642424 is kddi none!
50325642422
存在メッシュ：50325642422値85.86444605491788距離0.11554016109201981 km
50325642242
存在メッシュ：50325642242値107.89333786362087距離0.34662048011721447 km
50325642244
存在メッシュ：50325642244値96.87889553408513距離0.2310803581039872 km
50325642422
存在メッシュ：50325642422値85.86444605491788距離0.11554016109201981 km
50325642424
50325642424 is kddi none!
50325642224
存在メッシュ：50325642224値118.90777833072394距離0.46216058259375614 km
50325642242
存在メッシュ：50325642242値107.89333786362087距離0.34662048011721447 km
50325642224
存在メッシュ：50325642224値118.90777833072394距離0.46216058259375614 km
50325642224
存在メッシュ：50325642224値118.90777833072394距離0.46216058259375614 km
50325642222
存在メッシュ：50325642222値129.92222222222222距離0.5777007209917748 km
50325642242
存在メッシュ：50325642242値107.89333786362087距離0.34662048011721447 km
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
50325642424 is kddi none!
50325642422
存在メッシュ：50325642422値82.96631

33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値92.38333333333334距離0.0 km
50325642422
50325642422 is kddi none!
50325642242
存在メッシュ：50325642242値137.00201869590015距離0.34662048011721447 km
50325642244
存在メッシュ：50325642244値122.12912846991739距離0.2310803581039872 km
50325642422
50325642422 is kddi none!
50325642424
存在メッシュ：50325642424値92.38333333333334距離0.0 km
50325642224
存在メッシュ：50325642224値151.87490640702507距離0.46216058259375614 km
50325642242
存在メッシュ：50325642242値137.00201869590015距離0.34662048011721447 km
50325642224
存在メッシュ：50325642224値151.87490640702507距離0.46216058259375614 km
50325642224
存在メッシュ：50325642224値151.87490640702507距離0.46216058259375614 km
50325642222
存在メッシュ：50325642222値166.7477987421384距離0.5777007209917748 km
50325642242
存在メッシュ：50325642242値137.00201869590015距離0.34662048011721447 km
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値87.53333333333333距離0.0 km
50325642422
50325642422 is kddi none!
50325642242
存在

33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値73.81666666666666距離0.0 km
50325642422
存在メッシュ：50325642422値84.75407567328355距離0.11554016109201981 km
50325642242
存在メッシュ：50325642242値106.62889338749076距離0.34662048011721447 km
50325642244
存在メッシュ：50325642244値95.6914880802距離0.2310803581039872 km
50325642422
存在メッシュ：50325642422値84.75407567328355距離0.11554016109201981 km
50325642424
存在メッシュ：50325642424値73.81666666666666距離0.0 km
50325642224
存在メッシュ：50325642224値117.56629684537506距離0.46216058259375614 km
50325642242
存在メッシュ：50325642242値106.62889338749076距離0.34662048011721447 km
50325642224
存在メッシュ：50325642224値117.56629684537506距離0.46216058259375614 km
50325642224
存在メッシュ：50325642224値117.56629684537506距離0.46216058259375614 km
50325642222
存在メッシュ：50325642222値128.5037037037037距離0.5777007209917748 km
50325642242
存在メッシュ：50325642242値106.62889338749076距離0.34662048011721447 km
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値73.2距離0.0 km


...

33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値51.3距離0.0 km
50325642422
存在メッシュ：50325642422値59.07111224736161距離0.11554016109201981 km
50325642242
存在メッシュ：50325642242値74.61333652962418距離0.34662048011721447 km
50325642244
50325642244 is kddi none!
50325642422
存在メッシュ：50325642422値59.07111224736161距離0.11554016109201981 km
50325642424
存在メッシュ：50325642424値51.3距離0.0 km
50325642224
存在メッシュ：50325642224値82.38444483456908距離0.46216058259375614 km
50325642242
存在メッシュ：50325642242値74.61333652962418距離0.34662048011721447 km
50325642224
存在メッシュ：50325642224値82.38444483456908距離0.46216058259375614 km
50325642224
存在メッシュ：50325642224値82.38444483456908距離0.46216058259375614 km
50325642222
存在メッシュ：50325642222値90.15555555555555距離0.5777007209917748 km
50325642242
存在メッシュ：50325642242値74.61333652962418距離0.34662048011721447 km
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値61.016666666666666距離0.0 km
50325642422
存在メッシュ：50325642422値69.60928697065273

33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値72.11666666666666距離0.0 km
50325642422
50325642422 is kddi none!
50325642242
50325642242 is kddi none!
50325642244
50325642244 is kddi none!
50325642422
50325642422 is kddi none!
50325642424
存在メッシュ：50325642424値72.11666666666666距離0.0 km
50325642224
存在メッシュ：50325642224値114.19400052809064距離0.46216058259375614 km
50325642242
50325642242 is kddi none!
50325642224
存在メッシュ：50325642224値114.19400052809064距離0.46216058259375614 km
50325642224
存在メッシュ：50325642224値114.19400052809064距離0.46216058259375614 km
50325642222
存在メッシュ：50325642222値124.71333333333332距離0.5777007209917748 km
50325642242
50325642242 is kddi none!
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値75.08333333333333距離0.0 km
50325642422
50325642422 is kddi none!
50325642242
50325642242 is kddi none!
50325642244
50325642244 is kddi none!
50325642422
50325642422 is kddi none!
50325642424
存在メッシュ：50325642424値75.08333333

33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値19.633333333333333距離0.0 km
50325642422
存在メッシュ：50325642422値22.347755498929118距離0.11554016109201981 km
50325642242
50325642242 is kddi none!
50325642244
50325642244 is kddi none!
50325642422
存在メッシュ：50325642422値22.347755498929118距離0.11554016109201981 km
50325642424
存在メッシュ：50325642424値19.633333333333333距離0.0 km
50325642224
存在メッシュ：50325642224値30.49102054443243距離0.46216058259375614 km
50325642242
50325642242 is kddi none!
50325642224
存在メッシュ：50325642224値30.49102054443243距離0.46216058259375614 km
50325642224
存在メッシュ：50325642224値30.49102054443243距離0.46216058259375614 km
50325642222
存在メッシュ：50325642222値33.20544217687075距離0.5777007209917748 km
50325642242
50325642242 is kddi none!
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値26.983333333333334距離0.0 km
50325642422
存在メッシュ：50325642422値30.750833884197128距離0.11554016109201981 km
50325642242
50325642242 is kddi none!
50325642244

33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
50325642424 is kddi none!
50325642422
存在メッシュ：50325642422値95.52182010773132距離0.11554016109201981 km
50325642242
存在メッシュ：50325642242値121.86545996307902距離0.34662048011721447 km
50325642244
50325642244 is kddi none!
50325642422
存在メッシュ：50325642422値95.52182010773132距離0.11554016109201981 km
50325642424
50325642424 is kddi none!
50325642224
50325642224 is kddi none!
50325642242
存在メッシュ：50325642242値121.86545996307902距離0.34662048011721447 km
50325642224
50325642224 is kddi none!
50325642224
50325642224 is kddi none!
50325642222
存在メッシュ：50325642222値148.2090909090909距離0.5777007209917748 km
50325642242
存在メッシュ：50325642242値121.86545996307902距離0.34662048011721447 km
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値73.61666666666666距離0.0 km
50325642422
存在メッシュ：50325642422値84.76916829732585距離0.11554016109201981 km
50325642242
存在メッシュ：50325642242値107.07417125373703距離0.34662048011721447 km
50325642244
5032

33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値73.53333333333333距離0.0 km
50325642422
50325642422 is kddi none!
50325642242
存在メッシュ：50325642242値106.47706666352428距離0.34662048011721447 km
50325642244
存在メッシュ：50325642244値95.49582583420478距離0.2310803581039872 km
50325642422
50325642422 is kddi none!
50325642424
存在メッシュ：50325642424値73.53333333333333距離0.0 km
50325642224
存在メッシュ：50325642224値117.45830563602516距離0.46216058259375614 km
50325642242
存在メッシュ：50325642242値106.47706666352428距離0.34662048011721447 km
50325642224
存在メッシュ：50325642224値117.45830563602516距離0.46216058259375614 km
50325642224
存在メッシュ：50325642224値117.45830563602516距離0.46216058259375614 km
50325642222
存在メッシュ：50325642222値128.43954802259887距離0.5777007209917748 km
50325642242
存在メッシュ：50325642242値106.47706666352428距離0.34662048011721447 km
33.7838541665:33.7890625000:132.786718750:132.786718750
50325642424
存在メッシュ：50325642424値70.98333333333333距離0.0 km
50325642422
50325642422 is kddi none!
50325642242
存在

In [68]:
connection.close()

In [58]:
#CreateGeojsonFile